# 🎯 CaseGenius Enhanced: AI Case Study Generator with Ambiguity Resolution

**New Feature:** Intelligent ambiguity detection and interactive clarification

**Flow:**
1. Enter case brief
2. AI detects ambiguities
3. Configure which questions to ask (max 3)
4. Answer questions
5. Generate comprehensive case study

**Time:** 10-15 minutes | **Cost:** ~$4-5

---

## 🔧 Setup

In [ ]:
!pip install -q anthropic>=0.40.0

import anthropic
import json
import re
from typing import Dict, List, Optional, Tuple
from datetime import datetime
import time

In [ ]:
# API Key Setup
ANTHROPIC_API_KEY = ""  # ← Paste your key here

try:
    from google.colab import userdata
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    print("✅ Using API key from Colab secrets")
except:
    if not ANTHROPIC_API_KEY:
        raise ValueError("Please set ANTHROPIC_API_KEY")
    print("✅ Using API key from code")

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
MODEL = "claude-sonnet-4-20250514"
print(f"🤖 Model: {MODEL}")

## 📝 Step 1: Enter Your Case Brief

In [ ]:
# Example case brief with intentional ambiguities for demo
CASE_BRIEF = """
Create a comprehensive case study for a sub-brand launch.

Company: L'Oréal

Analysis Required:
1. CapEx & OpEx comparison with competitors
2. Market modeling and sizing
3. P&L projections
4. Business Canvas
5. Competitor Analysis
6. Market gap identification

The sub-brand should leverage the halo effect from L'Oréal's existing portfolio.
Focus on financial health and CapEx efficiency.

Timeline: Launch in the next fiscal year.
Target market: Should focus on high-growth segments.
"""

print("📋 Case Brief Loaded")
print(f"Length: {len(CASE_BRIEF)} characters")

## 🔍 Step 2: AI Detects Ambiguities

The system will analyze your brief and identify unclear references.

In [ ]:
class AmbiguityDetector:
    """Detects ambiguous terms in case briefs using single LLM call"""
    
    def __init__(self, client: anthropic.Anthropic):
        self.client = client
    
    def detect(self, brief: str) -> List[Dict]:
        """Single LLM call to detect all ambiguities"""
        print("\n🔍 Analyzing brief for ambiguities...")
        
        prompt = f"""Analyze this case study brief and identify ambiguous terms or references that need clarification.

Case Brief:
{brief}

Identify ambiguities in these categories:
1. TIME REFERENCES: "this year", "next quarter", "recently" (which specific date/period?)
2. VAGUE DESCRIPTORS: "high-growth", "major competitors", "key markets" (which specific ones?)
3. UNDEFINED SCOPE: "sub-brand" without category, "target market" without demographics
4. IMPLICIT ASSUMPTIONS: "fiscal year" (which company's fiscal calendar?)

For each ambiguity found, provide:
- term: The ambiguous phrase
- category: TIME/SCOPE/DESCRIPTOR/ASSUMPTION
- question: Clear question to resolve it
- options: 3-4 multiple choice options
- importance: HIGH/MEDIUM/LOW (how critical is this to the analysis?)

Return ONLY a JSON array. Example:
[
  {{
    "term": "next fiscal year",
    "category": "TIME",
    "question": "Which fiscal year should the launch target?",
    "options": ["FY 2026 (Jan-Dec 2026)", "FY 2027 (Jan-Dec 2027)", "L'Oréal FY 2026 (Jul 2025-Jun 2026)"],
    "importance": "HIGH"
  }}
]

If no ambiguities, return []."""
        
        try:
            response = self.client.messages.create(
                model=MODEL,
                max_tokens=4000,
                temperature=0,
                messages=[{"role": "user", "content": prompt}]
            )
            
            content = response.content[0].text
            
            # Extract JSON (remove markdown fences if present)
            json_match = re.search(r'\[.*\]', content, re.DOTALL)
            if json_match:
                ambiguities = json.loads(json_match.group())
                print(f"   ✅ Found {len(ambiguities)} potential ambiguities")
                return ambiguities
            else:
                print("   ✅ No ambiguities detected")
                return []
                
        except Exception as e:
            print(f"   ⚠️ Detection failed: {e}")
            return []

# Run detection
detector = AmbiguityDetector(client)
detected_ambiguities = detector.detect(CASE_BRIEF)

# Display results
if detected_ambiguities:
    print("\n📊 Detected Ambiguities:\n")
    for i, amb in enumerate(detected_ambiguities, 1):
        print(f"{i}. [{amb['importance']}] {amb['term']}")
        print(f"   Category: {amb['category']}")
        print(f"   Question: {amb['question']}")
        print()
else:
    print("\n✅ No ambiguities detected. Proceeding with analysis...")

## ⚙️ Step 3: Configure Which Questions to Ask

**Max 3 questions** - Select the most important ones to resolve.

In [ ]:
# CONFIGURABLE: Choose which ambiguities to ask about
# Edit the indices below (1-based indexing)

if detected_ambiguities:
    # Default: Ask about HIGH importance items, up to 3
    high_importance = [i for i, amb in enumerate(detected_ambiguities) if amb['importance'] == 'HIGH']
    
    # Auto-select top 3 high-importance, or all if fewer
    QUESTIONS_TO_ASK = high_importance[:3]
    
    print("🎛️ Configuration: Questions to Ask\n")
    print("AUTO-SELECTED (based on importance):")
    for idx in QUESTIONS_TO_ASK:
        amb = detected_ambiguities[idx]
        print(f"  [{idx+1}] {amb['term']} - {amb['importance']}")
    
    print("\n💡 To customize:")
    print("   Edit QUESTIONS_TO_ASK = [0, 2, 4] (change indices)")
    print("   Then re-run this cell")
    
else:
    QUESTIONS_TO_ASK = []
    print("No questions to configure.")

## 💬 Step 4: Answer Clarifying Questions

**Instructions:** For each question below, enter the number of your choice (e.g., 1, 2, or 3)

In [ ]:
# Interactive Q&A
user_answers = {}

if QUESTIONS_TO_ASK:
    print("="*60)
    print("📋 CLARIFYING QUESTIONS")
    print("="*60)
    print()
    
    for idx in QUESTIONS_TO_ASK:
        amb = detected_ambiguities[idx]
        
        print(f"\n❓ Question {idx+1}: {amb['question']}")
        print(f"   (Ambiguous term: '{amb['term']}')\n")
        
        for i, option in enumerate(amb['options'], 1):
            print(f"   {i}. {option}")
        
        # Text input (pause execution until answered)
        while True:
            answer_num = input(f"\n   Your choice (1-{len(amb['options'])}): ").strip()
            try:
                answer_num = int(answer_num)
                if 1 <= answer_num <= len(amb['options']):
                    user_answers[amb['term']] = amb['options'][answer_num - 1]
                    print(f"   ✅ Recorded: {amb['options'][answer_num - 1]}")
                    break
                else:
                    print(f"   ❌ Please enter a number between 1 and {len(amb['options'])}")
            except ValueError:
                print("   ❌ Please enter a valid number")
    
    print("\n" + "="*60)
    print("✅ ALL QUESTIONS ANSWERED")
    print("="*60)
    print("\nYour Clarifications:")
    for term, answer in user_answers.items():
        print(f"  • {term} → {answer}")
    
else:
    print("✅ No questions to answer. Proceeding with original brief.")

## 🔄 Step 5: Enrich Brief with Clarifications

In [ ]:
def enrich_brief(original_brief: str, clarifications: Dict[str, str]) -> str:
    """Add clarifications to brief as structured context"""
    
    if not clarifications:
        return original_brief
    
    enriched = original_brief + "\n\n" + "="*60 + "\n"
    enriched += "CLARIFICATIONS (resolved ambiguities):\n\n"
    
    for term, resolution in clarifications.items():
        enriched += f"• '{term}' means: {resolution}\n"
    
    enriched += "="*60
    
    return enriched

# Create enriched brief
ENRICHED_BRIEF = enrich_brief(CASE_BRIEF, user_answers)

print("📄 Enriched Brief Preview:\n")
print(ENRICHED_BRIEF[:500] + "...\n")
print(f"✅ Brief enriched with {len(user_answers)} clarifications")

## 🤖 Step 6: Generate Case Study

Now running the full 6-agent analysis with clarified context...

In [ ]:
# Cost tracking
class CostGuard:
    def __init__(self, max_cost_usd=10):
        self.total_input_tokens = 0
        self.total_output_tokens = 0
        self.max_cost = max_cost_usd
        self.input_price = 3 / 1_000_000
        self.output_price = 15 / 1_000_000
    
    def track(self, usage):
        self.total_input_tokens += usage.input_tokens
        self.total_output_tokens += usage.output_tokens
        cost = (self.total_input_tokens * self.input_price + 
                self.total_output_tokens * self.output_price)
        if cost > self.max_cost:
            raise Exception(f"Budget exceeded: ${cost:.2f}")
        return cost
    
    def report(self):
        cost = (self.total_input_tokens * self.input_price + 
                self.total_output_tokens * self.output_price)
        print(f"\n💰 Total API Cost: ${cost:.2f}")
        print(f"   Input tokens: {self.total_input_tokens:,}")
        print(f"   Output tokens: {self.total_output_tokens:,}")
        return cost

cost_guard = CostGuard(max_cost_usd=10)

In [ ]:
# Simplified agent for demo
class SimplifiedAgent:
    def __init__(self, name: str, role: str):
        self.name = name
        self.role = role
    
    def run(self, brief: str, context: Optional[str] = None) -> str:
        print(f"\n🔄 {self.name}...")
        
        system = f"""You are a {self.role} in management consulting.
Produce client-ready analysis with:
- Data-driven insights (cite sources)
- Structured frameworks
- Financial rigor
- Clear communication

CRITICAL: Pay attention to any CLARIFICATIONS in the brief - these resolve ambiguities."""
        
        user_prompt = f"""Create {self.name} for this case.

Case Brief:
{brief}

{f'Prior Analysis Context: {context[:1000]}...' if context else ''}

Generate comprehensive, professional analysis (2-3 pages markdown)."""
        
        try:
            start = time.time()
            response = client.messages.create(
                model=MODEL,
                max_tokens=8000,
                system=system,
                messages=[{"role": "user", "content": user_prompt}],
                tools=[{"type": "web_search_20250305", "name": "web_search"}]
            )
            
            # Extract text
            text = "".join([b.text for b in response.content if hasattr(b, 'text')])
            
            cost = cost_guard.track(response.usage)
            print(f"   ✅ Done in {time.time()-start:.1f}s (${cost:.2f} total)")
            
            return text
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
            return f"# {self.name}\n\n[Generation failed]"

print("✅ Agent system ready")

In [ ]:
# Run analysis
print("="*60)
print("🚀 GENERATING CASE STUDY")
print("="*60)

results = {}

# Only generate 3 key documents for demo (cost control)
agents = [
    SimplifiedAgent("Executive Summary", "Senior Strategy Consultant"),
    SimplifiedAgent("CapEx & OpEx Analysis", "CFO-level Financial Analyst"),
    SimplifiedAgent("Market Modeling", "Market Research Analyst")
]

context = ""
for agent in agents:
    output = agent.run(ENRICHED_BRIEF, context)
    results[agent.name] = output
    context += f"\n\n{agent.name}:\n{output[:500]}..."

print("\n" + "="*60)
print("✅ GENERATION COMPLETE")
print("="*60)

cost_guard.report()

## 💾 Step 7: Save Results

In [ ]:
import os

output_dir = f"case_study_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(output_dir, exist_ok=True)

# Save documents
file_map = {
    'Executive Summary': '01_Executive_Summary.md',
    'CapEx & OpEx Analysis': '02_CapEx_Analysis.md',
    'Market Modeling': '03_Market_Modeling.md'
}

for name, filename in file_map.items():
    filepath = os.path.join(output_dir, filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(results[name])
    print(f"✅ Saved: {filename}")

# Save clarifications log
clarifications_log = {
    'timestamp': datetime.now().isoformat(),
    'detected_ambiguities': detected_ambiguities,
    'questions_asked': [detected_ambiguities[i] for i in QUESTIONS_TO_ASK],
    'user_answers': user_answers
}

with open(os.path.join(output_dir, 'clarifications.json'), 'w') as f:
    json.dump(clarifications_log, f, indent=2)

print(f"\n📁 All files saved to: {output_dir}/")
print("\n📥 Download from Files panel (left sidebar)")

## 📊 Demo Summary

In [ ]:
print("\n" + "="*60)
print("📊 AMBIGUITY RESOLUTION DEMO SUMMARY")
print("="*60)

print(f"\n1️⃣ Ambiguities Detected: {len(detected_ambiguities)}")
if detected_ambiguities:
    for amb in detected_ambiguities:
        print(f"   • {amb['term']} ({amb['importance']})")

print(f"\n2️⃣ Questions Asked: {len(QUESTIONS_TO_ASK)}")
for term, answer in user_answers.items():
    print(f"   • {term}\n     → {answer}")

print(f"\n3️⃣ Documents Generated: {len(results)}")
for name in results.keys():
    word_count = len(results[name].split())
    print(f"   • {name}: {word_count:,} words")

final_cost = cost_guard.report()

print("\n✨ Key Innovation:")
print("   Human-in-the-loop clarification BEFORE expensive analysis")
print("   Prevents wasted API calls on ambiguous briefs")
print("   Ensures analysis matches PM's actual intent")

print("\n" + "="*60)

---

## 🎯 What Just Happened?

### Enhanced Workflow:

1. ✅ **Detected ambiguities** in your brief (single LLM call)
2. ✅ **Configured questions** (you chose top 3 by importance)
3. ✅ **Answered clarifications** (interactive multiple choice)
4. ✅ **Enriched brief** with your answers
5. ✅ **Generated analysis** using clarified context

### Why This Matters:

**Before (Original CaseGenius):**
- PM submits brief with "next fiscal year"
- AI guesses which year (might be wrong)
- PM wastes time fixing incorrect assumptions

**After (Enhanced Version):**
- AI detects "next fiscal year" is ambiguous
- Asks: "Which fiscal year? 2026, 2027, or L'Oréal FY 2026?"
- PM clarifies once
- All 6 documents use correct year

### Cost Efficiency:

- **Ambiguity detection:** 1 LLM call (~$0.05)
- **Prevents:** Re-running entire analysis ($4) due to wrong assumptions
- **ROI:** 80:1 cost savings when avoiding rework

---

## 🔄 Try It Again

Want to test with your own case brief?

1. Edit the `CASE_BRIEF` cell at the top
2. Run all cells again
3. Answer the new clarifying questions
4. Get a tailored case study

---

*Generated by **CaseGenius Enhanced** - Ambiguity-Aware AI Assistant*
